<a href="https://colab.research.google.com/github/shahzaibkhan/learning-langchain/blob/main/Learning_LangChain_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Leanring Models with LangChain**

A model is a program which is trained to complete a specific task.

Note:
- For current case, we be using Language Models.
- We are not going to train a model
- We will only be using trained models.

These pre-trained models are trained on large amounts of data and require a lot of compute to run and thus are called Large Language Models (LLM).



## **LLM (Large Language Model)**

Now let's talk about LLM. LLM are trained to do language tasks like text generation. There are various LLM in the market but we are going to cover only OpenAI.

So lets dive in directly learning how to use models with langchain.

Let's install necessary libraries:


In [ ]:
!pip install langchain
!pip install openai

# The tiktoken package is a Byte Pair Encoding tokenizer. It is used with the OpenAI models and breaks down text into tokens.
# This is used because the provided strings are sometimes a bit long for the specified OpenAI model.
# So, it splits the text and encodes them into tokens. Now, let’s work on the main project.
!pip install tiktoken

In [ ]:
# Lets set the OPEN API KEY
import os
os.environ["OPENAI_API_KEY"]="YOUR_API_KEY”

To use a model here is how it is done:

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=1)
print(llm("Write a positive quote about life"))

### **Estimating number of tokens**

OpenAI models have a context length limiting the size of input data which can be sent to the model. Thus we need to make sure the input text is below that limit before sending to the model. We can do that token calculation using the code below:

In [ ]:
llm.get_num_tokens("Write a positive quote about life")

### **Streaming**

Streaming is an important concept in LLM which allows you to display output on the go instead of waiting for the full output. Even in the ChatGPT interface you will see content streamed instead of waiting till entire output is generated

Here is a code example for the same. We handle streaming in langchain using a callback handler:

In [ ]:
from langchain.llms import OpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = OpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0)
resp = llm("Write me a poem about life")

## **Chat Models**

The famous ChatGPT model GPT-3.5 comes under this. The main difference between the previous LLM models and the Chat Models are:

- Chat Models are 10x cheaper for api calls
- You can hold a conversation with a chat model like you can with a human which is not possible with the previous LLMs

Since Chat Models can hold a conversation they take a list of chat messages as input instead of plain text like a LLM

Now let's discuss how we can use these Chat Models.
Let's do the necessary imports first:

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
#Instead of using a OpenAI class we will be using a ChatOpenAI class to create our chat LLM
#Replace your OpenAI key here

chat = ChatOpenAI(temperature=0, openai_api_key="openai-key")


**Input is a bunch of messages.**

Messages are classified into 3 types

1. **System Message** - This is an initial prompt sent to the model to control the behavior of the model.
2. **Human Message** - Input message of the user
3. **AI Message** - Message response given by ChatGPT

ChatGPT needs a list of all these messages in the conversation to be able to understand the content and converse further.

Now let's see an example where we define the system message and the message input of the user and pass to the chat model. The output generated will be an AI message.

We are using a System Prompt to let the model do the task of paraphrasing. This technique of providing the model a prompt to make it perform a task is called Prompt Engineering.

In [ ]:
messages = [
    SystemMessage(content="Consider yourself as a expert in English Language who can help paraphrase sentences"),
    HumanMessage(content="I love programming.")
]
chat(messages)

### **Templates in Chat Models**

Earlier, we defined a system message with input variable task. This task can be dynamically change to do various tasks. For this example we will follow the task of paraphrasing.



In [ ]:
template="You are a helpful assistant that {task}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat(chat_prompt.format_prompt(task="paraphrases the sentence", text="I love programming.").to_messages())

### **Chaining in Chat Models**
Chaining multiple tasks for LLMs can be achieved with Chat Models as well. Here is an example:


In [ ]:
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(task="paraphrases the sentence", text="I love programming.")

### **Streaming with Chat Models**
As disucussed above how streaming can be useful in LLMs. Now let's see how we can do the same with Chat Models.

In [ ]:
# Replace openai-key with your own key

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
chat = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0, openai_api_key="openai-key")
resp = chat([HumanMessage(content="Write me a poem about life")])

# **Embedding models**

First we need to understand what is an embedding. An embedding is generally associated with a piece of text and it represents the properties of text.

Just to give an example, let's consider the words good, best, bad. If we find the embeddings of these words we observe that embeddings of good and best are close while embedding of bad is far. The reason being embedding of a word has knowledge of the meaning of the word. Thus words with similar meanings have similar embeddings.

Embeddings also have an **interesting** as can be seen below. Let's consider E(x) as Embedding of word x

E(king) - E(male) + E(female) ~= E(queen)

What this represents is if we subtract the embedding of word male from word king and add the embedding of word female it will be quite close to embedding of word queen. As humans we can understand this intuitively as removing male gender and adding female gender to king makes it a queen but now machines have the capability to understand such complex relations.

Now that we have an idea of what is embeddings, the task of a embeddings model is to create these embeddings for the text input provided. A model which generates embedding which can show properties like the ones we discussed above and more is considered a good model.

Once these embeddings are generated, we can use it to perform tasks like semantic search similar to how apps like Chatbase, PDF.ai, SiteGPT work. You can creating embeddings for all your documents or webpages and when user asks a query you can fetch the relevant pieces and send to the user

Now let's discuss it with the help of an example:


In [ ]:
# Replace openai-key with your own key

from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(openai_api_key="openai-key")
strings = ["This is for demonstration.", "This string is also for demonstration.", "This is another demo string.", "This one is last string."]
result = model.embed_documents(strings)

# To identify if sentences are similar, we can calculate the distance between these vectors.
# If the distance is small, then the words are of similar meanings
print(result)